# Lab 9 - HITS Algirhtm
## Advait Deochakke 20BCE1143

In [19]:
# assuming outlink data is processed into a NetworkX graph G
import numpy as np
import networkx as nx
from math import sqrt
from math import pow

In [28]:
def get_node_dict(G):
    node_dict = {node: i for i, node in enumerate(G.nodes())}
    return node_dict

def get_predec_dict(G, node):
    node_dict = {node1: i for i, node1 in enumerate(G.predecessors(node))}
    return node_dict
    
def get_succ_dict(G, node):
    nodes = list(G.nodes())
    #node_index = nodes.index(node)
    succ_dict = {succ: nodes.index(succ) for succ in G.successors(node)}
    return succ_dict


In [29]:
def hits(G, max_iter=100, conv_tol=1e-6):
    n = G.number_of_nodes()
    hubs = np.ones(n) # set all hub scores as 1
    authorities = np.ones(n) # set all authority scores as 1
    node_dict = get_node_dict(G)
    #print(node_dict)
    for iterations in range(max_iter): # iteratae
        norm_auth =0
        norm_hub = 0
        old_hubs = hubs.copy()
        old_authorities = authorities.copy()
    #     idx = 0
        # update hubs and authorities
        for node in G.nodes():
            node_idx = node_dict[node]
            predec_dict = get_predec_dict(G, node)
            predec_idx = [node_dict[node1] for node1 in predec_dict]
            authorities[node_idx] = sum(old_hubs[k] for k in predec_idx)
            norm_auth += pow(authorities[node_idx], 2)
            #print("authority for node", node, "is", authorities[node_idx])# graph things
        norm_auth = sqrt(norm_auth)
        
        for node in G.nodes():
            node_idx = node_dict[node]
            succ_dict = get_succ_dict(G, node)
            indices = [succ_dict[n] for n in G.successors(node)]
            hubs[node_idx] = sum(old_authorities[k] for k in indices)
            norm_hub += pow(hubs[node_idx], 2)
            #print("hubs for node", node, "is", hubs[node_idx])
        norm_hub = sqrt(norm_hub)
        #hubs = np.array(np.split(hubs, len(G)))
        #hubs = np.array(list(get_succ_dict(G, node1).values()) for node1 in G.nodes())
        #hubs = np.sum(hubs, axis=1)
        
        # normalize with method
        hubs = hubs/norm_hub
        authorities = authorities/norm_auth
        #norm = np.linalg.norm(hubs)
        # hubs /= norm
        # authorities /= norm
        
        # Check for convergence
        if np.max(np.abs(hubs - old_hubs)) < conv_tol and np.max(np.abs(authorities - old_authorities)) < conv_tol:
            break
    return hubs, authorities

In [30]:

G = nx.DiGraph()

# Add the nodes
G.add_nodes_from(['A', 'B', 'C', 'D', 'E', 'F'])

# Add the edges
G.add_edges_from([('A', 'B'), ('A', 'C'), ('A', 'D'),
                  ('B', 'A'), ('B', 'C'), ('B', 'D'),
                  ('C', 'B'), ('C', 'D'),
                  ('D', 'A'), ('D', 'D'), ('D', 'E'),
                  ('E', 'B'), ('E', 'C'), ('E', 'D'),
                  ('F', 'A')])
# it takes you about two hours to realize that working with a complete weighted, directed graph with
# base weight as zero to show no edge and weight as 1 to show edge
# is so much easier to work with than whatever i was tyring to do above

print(G)

DiGraph with 6 nodes and 15 edges


In [31]:
hubs_hard, authorities_hard = hits(G)
print("Hubs:", hubs_hard)
print("Authorities:", authorities_hard)

Hubs: [0.51218386 0.46336098 0.36194982 0.34829915 0.51218386 0.09117025]
Authorities: [0.2868994  0.44053938 0.4727658  0.69846677 0.11068176 0.        ]


In [6]:
hubs, authorities = nx.hits(G)

print("Hubs:", hubs)
print("Authorities:", authorities)

Hubs: {'A': 0.22374463599524844, 'B': 0.2024161791536921, 'C': 0.15811574414224222, 'D': 0.15215192574521555, 'E': 0.22374463599524844, 'F': 0.039826878968353255}
Authorities: {'A': 0.14278134433896675, 'B': 0.21924492419750571, 'C': 0.23528284537924313, 'D': 0.3476078921561056, 'E': 0.05508299392817882, 'F': -0.0}


c:\Users\advai\anaconda3\lib\site-packages\networkx\algorithms\link_analysis\hits_alg.py:78: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(G, nodelist=list(G), dtype=float)


For whatever reason, the hard-coded Hubs and Authorities are around double, or rather

perfectly in ratio with the nx.hits version. Im not sure where im going wrong, but ratio wise we good